---
# INFO-H600 - Computing Foundations of Data Sciences

## Team 14 : 

Roman Lešický, Theo Abraham, Kevin Straatman, Lara Hansen, Grégoire Van den Eynde and Nicolas Roux

Version of python : `TBD` | packaged by conda-forge (`to be detailed`)

---

# Library:



---

# Overview of the project :

# `WIP`

---

# Step 1:

`TBD`